In [1]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from keras import models
from keras import layers
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
np.random.seed(42)

Wall time: 5.96 s


## Reading cleaned data

In [2]:
df=pd.read_csv('bangla_emotions_dataset_11927.csv',encoding='utf-8')
df.head()

,texts,classes,labels,ctexts,no_words
0,মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...,sadness,1,মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...,21
1,অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে কেননা আমার...,sadness,1,অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে কেননা আমার...,30
2,আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...,disgust,3,আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...,23
3,মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...,sadness,1,মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...,26
4,এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...,fear,5,এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...,30


In [3]:
df=df.drop(['texts','classes','no_words'],axis=1)
df.head()

,labels,ctexts
0,1,মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...
1,1,অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে কেননা আমার...
2,3,আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...
3,1,মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...
4,5,এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11927 entries, 0 to 11926
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   labels  11927 non-null  int64 
 1   ctexts  11927 non-null  object
dtypes: int64(1), object(1)
memory usage: 186.5+ KB


In [5]:
df

,labels,ctexts
0,1,মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...
1,1,অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে কেননা আমার...
2,3,আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...
3,1,মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...
4,5,এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...
...,...,...
11922,1,চারপাশের এত অনিয়ম দেখে যখন ভীষণভাবে ডিপ্রেশনে...
11923,2,আট মাসে অর্ধেক রিজার্ভ শেষ
11924,0,বাংলাদেশের একমাত্র অনুষ্ঠান ইত্যাদি যার জনপ্রি...
11925,2,বাহ এই একটা অনুষ্ঠান ই ঠিক আগের মতোবাংলার দর্শ...


In [6]:
df.labels.value_counts()

0    2654
1    2477
3    2229
4    2047
2    1266
5    1254
Name: labels, dtype: int64

In [7]:
x_train, x_test, y_train, y_test = train_test_split(df['ctexts'].values, df['labels'].values, test_size=0.25,shuffle=True)

In [8]:
print('Train: ',x_train.shape,y_train.shape)
print('Test: ', x_test.shape, y_test.shape)

Train:  (8945,) (8945,)
Test:  (2982,) (2982,)


In [9]:
import collections, numpy
a= collections.Counter(y_train)
a

Counter({1: 1871, 2: 922, 4: 1544, 0: 1972, 3: 1702, 5: 934})

In [10]:
a= collections.Counter(y_test)
a

Counter({4: 503, 2: 344, 5: 320, 0: 682, 1: 606, 3: 527})

## ML Models

In [11]:
%%time
lr_model = LogisticRegressionCV(class_weight='balanced', max_iter=400)
dt_model = DecisionTreeClassifier(class_weight='balanced')
rf_model = RandomForestClassifier(class_weight='balanced')
mnb_model = MultinomialNB()
svm_model = SVC(class_weight='balanced')
knn_model = KNeighborsClassifier()
ada_model = AdaBoostClassifier()
model_names = ['Logistic Regression','Decision Tree','Random Forest','Naive Bayes', 'SVM', 'KNN', 'AdaBoost']
ml_models = [lr_model,dt_model,rf_model,mnb_model, svm_model, knn_model, ada_model]

Wall time: 996 µs


In [12]:
%%time
def model_train(model, x_train, y_train):
  pipe = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000)),  
        ('Transformer', TfidfTransformer()),  
        ('M', model)
    ])
  pipe.fit(x_train, y_train)
  return pipe

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,x_train, y_train)
    trained_ml_models.append(tt)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Wall time: 3min 43s


### Performance on validation set

### Performance on test set

In [13]:
pr_list = []
re_list = []
f1_list = []
acc_list = []

def print_results(md, x_test, y_test, name):
    #print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
    # print(confusion_matrix(true,pred))
    # print(classification_report(true,pred))
    # print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
    # print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
    # print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    acc_list.append((round(accuracy_score(y_test, pred_y),4)*100))

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, x_test, y_test, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':acc_list},
                                  index =model_names)
performance_matrix

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


,Precision,Recall,F1 Score,Accuracy
Logistic Regression,54.15,54.23,54.15,54.23
Decision Tree,39.41,39.30,39.29,39.30
Random Forest,51.51,50.13,49.14,50.13
Naive Bayes,58.72,50.44,47.44,50.44
SVM,56.88,53.66,53.57,53.66
KNN,43.92,33.57,34.16,33.57
AdaBoost,46.08,36.28,36.31,36.28


## Ensemble

In [14]:
whole_data = []

def make_whole_list(L):
  for i in L:
    whole_data.append(i)

make_whole_list(x_train)
#make_whole_list(val_data)
make_whole_list(x_test)
print(len(whole_data))


11927


In [15]:
tf = TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(),  max_features=20000)
tf.fit_transform(whole_data) ### Bit confusion arises here need to resolve
trained_texts = tf.transform(x_train)
#val_texts = tf.transform(val_data)
test_texts = tf.transform(x_test)

In [16]:
%%time
from sklearn.ensemble import VotingClassifier
LogReg_clf = LogisticRegressionCV(class_weight='balanced', max_iter=400)
rt_clf =   RandomForestClassifier(class_weight='balanced')
SVC_clf = SVC(class_weight='balanced')
mnb_model = MultinomialNB()


voting_clf = VotingClassifier(estimators=[('SVC', SVC_clf), ('LogReg', LogReg_clf),('RF', rt_clf),('mnb',mnb_model)], voting='hard')
voting_clf.fit(trained_texts, y_train)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Wall time: 3min 57s


VotingClassifier(estimators=[('SVC', SVC(class_weight='balanced')),
                             ('LogReg',
                              LogisticRegressionCV(class_weight='balanced',
                                                   max_iter=400)),
                             ('RF',
                              RandomForestClassifier(class_weight='balanced')),
                             ('mnb', MultinomialNB())])

### Val set

### Test set

In [18]:
preds = voting_clf.predict(test_texts)
true, pred = y_test, preds
print(confusion_matrix(true,pred))
print(classification_report(true,pred))
print("\nPrecison : ",round(precision_score(true, pred, average = 'weighted'),4)*100)
print("\nRecall : ",round(recall_score(true, pred,  average = 'weighted'),4)*100)
print("\nF1 : ",round(f1_score(true, pred,  average = 'weighted'),4)*100)
print("\nAccuracy : ", round(accuracy_score(true, pred),4)*100)

[[515  89   8  36  30   4]
 [ 94 371  20  65  44  12]
 [ 86  92  82  45  24  15]
 [ 51 105  15 287  63   6]
 [ 55 120  15  92 216   5]
 [ 37  78  14  17  22 152]]
              precision    recall  f1-score   support

           0       0.61      0.76      0.68       682
           1       0.43      0.61      0.51       606
           2       0.53      0.24      0.33       344
           3       0.53      0.54      0.54       527
           4       0.54      0.43      0.48       503
           5       0.78      0.47      0.59       320

    accuracy                           0.54      2982
   macro avg       0.57      0.51      0.52      2982
weighted avg       0.56      0.54      0.54      2982


Precison :  55.910000000000004

Recall :  54.43

F1 :  53.53

Accuracy :  54.43
